# <a>Classificação de Imagens - PyTorch e Transfer Learning</a>

Projeto realizado durante a Jornada Cientista de Dados da equipe <i>Let's Data</i>

# <a>Objeto de Estudo</a>

Iremos modelar um Produto Mínimo Viável que possa ser testado nas filiais de uma empresa do ramo de supermercados. O problema a ser solucionado é classificar produtos na hora de colocar no mostruário. Vamos começar com uma classificação simples de batatas, cenouras, tomates e limões.

Para isso iremos usar modelos pré-treinados para nos ajudar com a classificação de imagens através dos padrões.

<b>Criação e Separação de Bases:</b>

In [1]:
!pip install tbb

In [2]:
!pip install torch torchvision

  Using cached torchvision-0.18.1-cp310-cp310-win_amd64.whl (1.2 MB)
  Using cached torch-2.3.1-cp310-cp310-win_amd64.whl (159.8 MB)
  Using cached mkl-2021.4.0-py2.py3-none-win_amd64.whl (228.5 MB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
  Using cached intel_openmp-2021.4.0-py2.py3-none-win_amd64.whl (3.5 MB)
  Using cached tbb-2021.13.0-py3-none-win_amd64.whl (286 kB)
  Attempting uninstall: tbb
    Found existing installation: TBB 0.2


ERROR: Cannot uninstall 'TBB'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.


In [3]:
!pip install pillow

In [4]:
# Importando as bibliotecas necessárias:
from matplotlib import pyplot as plt
import numpy as np
import os
import PIL.Image

In [5]:
import time
import torch, torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

ModuleNotFoundError: No module named 'torchvision'

In [8]:
# Vamos separar as imagens em bases de treino, validação e teste
# Mas antes vamos deixar salva uma pasta raw com as originais

diretorio_base_imagens = 'C:\\data_science\\projeto-classificacao-imagens\\data\\raw'
pastas_com_nome_de_vegetais = os.listdir('C:\\data_science\\projeto-classificacao-imagens\\data\\raw')
pastas_com_nome_de_vegetais

['batata', 'cenoura', 'limao', 'tomate']

In [9]:
pastas_com_nome_de_vegetais

['batata', 'cenoura', 'limao', 'tomate']

In [11]:
# Vamos criar uma separação estratificada:
# Queremos 80% das imagens na base de treino, 10% na validação e 10% no teste

quantidade_por_label = {pasta: len(os.listdir(os.path.join(diretorio_base_imagens, pasta))) for pasta in pastas_com_nome_de_vegetais}
quantidade_por_label

{'batata': 146, 'cenoura': 181, 'limao': 111, 'tomate': 107}

In [12]:
# Criando as pastas de treino, validação e testes

diretorio_imagens_processadas = 'C:\\data_science\\projeto-classificacao-imagens\\data\\processed'

dir_treino = os.path.join(diretorio_imagens_processadas, 'treino')
dir_validacao = os.path.join(diretorio_imagens_processadas, 'validacao')
dir_teste = os.path.join(diretorio_imagens_processadas, 'teste')

if not os.path.exists(dir_treino):
    os.makedirs(dir_treino)

if not os.path.exists(dir_validacao):    
    os.makedirs(dir_validacao)

if not os.path.exists(dir_teste):
    os.makedirs(dir_teste)

In [13]:
import shutil
from sklearn.model_selection import train_test_split

# Criando uma pasta para cada classe (batata, cenoura, limao, tomate)
# dentro de treino, validação e teste

for classe in pastas_com_nome_de_vegetais:
    # os.path.join cria paths com os separadores corretos pra cada sistema operacional
    # barra normal, barra invertida, isso muda do Windows pro Linux/Mac
    dir_classe_treino = os.path.join(dir_treino, classe)
    dir_classe_validacao = os.path.join(dir_validacao, classe)
    dir_classe_teste = os.path.join(dir_teste, classe)
    
    # Efetivamente criando as pastas de treino, validação e teste
    # Testa primeiro se as pastas já não existem
    if not os.path.exists(dir_classe_treino):
        os.makedirs(dir_classe_treino)

    if not os.path.exists(dir_classe_validacao):
        os.makedirs(dir_classe_validacao)
    
    if not os.path.exists(dir_classe_teste):
        os.makedirs(dir_classe_teste)
    
    # caminho completo para a pasta com imagens originais
    pasta_classe = os.path.join(diretorio_base_imagens, classe)
    
    # listando todos os arquivos de imagem para essa classe
    arquivos_classe = os.listdir(pasta_classe)
    
    # separando 80% para treino e 20% para validação+teste
    treino, valid_teste = train_test_split(arquivos_classe, 
                                           shuffle=True, 
                                           test_size=0.2, 
                                           random_state=42)
    
    # separando os 20% da validação+teste em 10% para validação e 10% para teste
    validacao, teste = train_test_split(valid_teste, shuffle=True, test_size=0.5, random_state=42)
    
    # Não precisamos mais dessa lista temporária
    del valid_teste
    
    print(f'{classe} - treino: {len(treino)} - valid: {len(validacao)} - teste: {len(teste)} - total: {len(arquivos_classe)}')
    
    # Copiando os arquivos efetivamente para as pastas de treino, validação e teste
    for imagem_treino in treino:
        caminho_origem = os.path.join(diretorio_base_imagens, classe, imagem_treino)
        caminho_destino = os.path.join(dir_classe_treino, imagem_treino)

        shutil.copy(caminho_origem, caminho_destino)

    for imagem_validacao in validacao:
        caminho_origem = os.path.join(diretorio_base_imagens, classe, imagem_validacao)
        caminho_destino = os.path.join(dir_classe_validacao, imagem_validacao)

        shutil.copy(caminho_origem, caminho_destino)

    for imagem_teste in teste:
        caminho_origem = os.path.join(diretorio_base_imagens, classe, imagem_teste)
        caminho_destino = os.path.join(dir_classe_teste, imagem_teste)

        shutil.copy(caminho_origem, caminho_destino)  

batata - treino: 116 - valid: 15 - teste: 15 - total: 146
cenoura - treino: 144 - valid: 18 - teste: 19 - total: 181
limao - treino: 88 - valid: 11 - teste: 12 - total: 111
tomate - treino: 85 - valid: 11 - teste: 11 - total: 107
